[Link](https://www.kesci.com/home/competition/5c234c6626ba91002bfdfdd3)

# 导入工具

In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
import os
import json

TEMP_FOLDER = os.path.abspath('./temp/')
print(f"文件夹 '{TEMP_FOLDER}' 将被用来存储语料和临时性字典")

import logging
logging.basicConfig(format="%(asctime)s: %(levelname)s: %(message)s", level=logging.INFO)

import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
import pandas_profiling

import seaborn as sns
sns.set()
import matplotlib.pyplot as  plt
import matplotlib.patches as patches
plt.rcParams['font.sans-serif'] = ['SimHei']
%matplotlib inline

from sklearn.decomposition import FactorAnalysis, FastICA, PCA, TruncatedSVD
from sklearn.manifold import Isomap, TSNE
import umap

import xgboost as xgb
import lightgbm as lgb

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.preprocessing import Imputer, Normalizer, scale, StandardScaler

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.feature_selection import RFECV, SelectKBest, chi2
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, auc, roc_auc_score
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
print(tf.__version__)

文件夹 'D:\Github\NLP\Kaggle\提供银行精准营销解决方案_kesci\temp' 将被用来存储语料和临时性字典


Using TensorFlow backend.


2.0.0


In [4]:
train_df = pd.read_csv('data/train_set.csv')
test_df = pd.read_csv('data/test_set.csv')

In [3]:
train_df.tail(1)

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
25316,25317,52,management,married,tertiary,no,40,no,no,cellular,19,nov,1265,3,-1,0,unknown,1


In [4]:
test_df.tail(1)

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
10851,36169,34,blue-collar,married,primary,no,15841,yes,no,cellular,20,apr,423,5,-1,0,unknown


In [80]:
neg, pos = train_df.y.value_counts()
total = neg + pos
pos/total, neg/total

(0.11695698542481336, 0.8830430145751866)

In [73]:
train_df_copy = train_df.copy()
test_df_copy = test_df.copy()

In [74]:
train_y = train_df.y
train_df_copy.drop(['y'], axis=1, inplace=True)
full_x = pd.concat([train_df_copy, test_df_copy], ignore_index=True)

full_x.drop(['ID'], axis=1, inplace=True)

for column in full_x.columns:
    if full_x[column].dtype == 'object':
        full_x[column] = full_x[column].astype('category').cat.codes

In [75]:
train_x = full_x.iloc[:25317,:]
test_x = full_x.iloc[25317:,:]
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=.3, random_state=42)
train_x.shape, valid_x.shape, train_y.shape, valid_y.shape

((17721, 16), (7596, 16), (17721,), (7596,))

# 模型

In [15]:
kfold = StratifiedKFold(5)

In [76]:
LR = LogisticRegression()
lr_param_grid = {'penalty': ['l1','l2'],
                'max_iter': [100,300]}
gsLR = GridSearchCV(LR, param_grid=lr_param_grid, cv=kfold, scoring='accuracy', verbose=1)
gsLR.fit(train_x, train_y)
lr_best = gsLR.best_estimator_
gsLR.best_score_, lr_best

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    3.7s finished


(0.8914282489701484,
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=300,
                    multi_class='warn', n_jobs=None, penalty='l1',
                    random_state=None, solver='warn', tol=0.0001, verbose=0,
                    warm_start=False))

In [77]:
pred_y = lr_best.predict(valid_x)
print(classification_report(valid_y, pred_y))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      6712
           1       0.64      0.22      0.33       884

    accuracy                           0.89      7596
   macro avg       0.77      0.60      0.63      7596
weighted avg       0.87      0.89      0.87      7596



In [78]:
confusion_matrix(valid_y, pred_y)

array([[6605,  107],
       [ 691,  193]], dtype=int64)

In [79]:
roc_auc_score(valid_y, lr_best.predict_proba(valid_x)[:,1])

0.8783523735431645

## Class Weight

In [81]:
neg, pos = train_y.value_counts()
total = neg+pos
print(neg/total, pos/total)
weight_for_neg = (1/neg) * total / 2.
weight_for_pos = (1/pos) * total / 2.
print(weight_for_neg, weight_for_pos)
class_weight = {0:weight_for_neg, 1:weight_for_pos}

0.882794424693866 0.11720557530613397
0.5663832779340323 4.266008666345691


In [46]:
LR = LogisticRegression(class_weight=class_weight)
lr_param_grid = {'penalty': ['l1','l2'],
                'max_iter': [100,300]}
gsLR = GridSearchCV(LR, param_grid=lr_param_grid, cv=kfold, scoring='accuracy', verbose=1)
gsLR.fit(train_x, train_y)
lr_best = gsLR.best_estimator_
gsLR.best_score_, lr_best

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    3.8s finished


(0.7995034140285537, LogisticRegression(C=1.0,
                    class_weight={0: 0.5663832779340323, 1: 4.266008666345691},
                    dual=False, fit_intercept=True, intercept_scaling=1,
                    l1_ratio=None, max_iter=300, multi_class='warn', n_jobs=None,
                    penalty='l1', random_state=None, solver='warn', tol=0.0001,
                    verbose=0, warm_start=False))

In [47]:
pred_y = lr_best.predict(valid_x)
print(classification_report(valid_y, pred_y))

              precision    recall  f1-score   support

           0       0.97      0.81      0.88      6712
           1       0.35      0.80      0.49       884

    accuracy                           0.81      7596
   macro avg       0.66      0.80      0.69      7596
weighted avg       0.90      0.81      0.84      7596



In [48]:
confusion_matrix(valid_y, pred_y)

array([[5427, 1285],
       [ 180,  704]], dtype=int64)

In [49]:
roc_auc_score(valid_y, lr_best.predict_proba(valid_x)[:,1])

0.8762823995922747

In [45]:
GBC = GradientBoostingClassifier(n_estimators=300,learning_rate=.05, max_depth=8)
gb_param_grid = {'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1]}

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring=roc_auc_score, verbose = 1)

gsGBC.fit(train_x,train_y)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_, gsGBC.best_params_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed: 10.0min finished


(0.9076237232661813,
 {'learning_rate': 0.05,
  'loss': 'deviance',
  'max_depth': 8,
  'max_features': 0.3,
  'min_samples_leaf': 100,
  'n_estimators': 300})

In [10]:
gbc = GradientBoostingClassifier(learning_rate=.05, max_depth=8, n_estimators=300,
                                 max_features=.3, min_samples_leaf=100)
gbc.fit(train_x, train_y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=8,
                           max_features=0.3, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=100, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [75]:
pred_y = gbc.predict(valid_x)
print(classification_report(valid_y, pred_y))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      6712
           1       0.65      0.46      0.54       884

    accuracy                           0.91      7596
   macro avg       0.79      0.71      0.74      7596
weighted avg       0.90      0.91      0.90      7596



In [76]:
confusion_matrix(valid_y, pred_y)

array([[6490,  222],
       [ 478,  406]], dtype=int64)

In [12]:
prob = gbc.predict_proba(valid_x)
prob

array([[0.66916099, 0.33083901],
       [0.96986874, 0.03013126],
       [0.49987953, 0.50012047],
       ...,
       [0.96951013, 0.03048987],
       [0.9982488 , 0.0017512 ],
       [0.62586876, 0.37413124]])

In [19]:
valid_y.values

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [25]:
roc_auc_score(valid_y.values, prob[:,1])

0.9325579835399824

In [49]:
submission = gbc.predict_proba(test_x)[:,1]

In [50]:
submission = pd.DataFrame(submission, columns=['pred'])

In [53]:
submission = pd.concat([test_df.ID, submission], axis=1)

In [59]:
submission.to_csv('submission1.csv', index_label=False, index=False)

In [96]:
gbc = GradientBoostingClassifier(learning_rate=.05, max_depth=8, n_estimators=300,
                                 max_features=.3, min_samples_leaf=100)
gbc.fit(os_features, os_labels)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.05, loss='deviance', max_depth=8,
                           max_features=0.3, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=100, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=300,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [97]:
roc_auc_score(valid_y, gbc.predict_proba(valid_x)[:,1])

0.9205668984839741

In [ ]:
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [0.01, 0.1],
                  'C': [1, 10]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(train_x, train_y)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

In [71]:
SVMC = SVC(probability=True)
SVMC.fit(train_x, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [74]:
roc_auc_score(valid_y, SVMC.predict_proba(valid_x)[:,1])

0.7164861239948441

In [109]:
XGB = xgb.XGBClassifier(n_jobs=4)
xgb_param_grid = {'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8]
              }

gsXGB = GridSearchCV(XGB,param_grid = xgb_param_grid, cv=kfold, scoring='accuracy', verbose = 1)

gsXGB.fit(train_x,train_y)

XGB_best = gsXGB.best_estimator_

# Best score
gsXGB.best_score_, XGB_best

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  2.0min finished


(0.9061565374414536,
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
               max_depth=4, min_child_weight=1, missing=None, n_estimators=300,
               n_jobs=4, nthread=None, objective='binary:logistic',
               random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
               seed=None, silent=True, subsample=1))

In [110]:
roc_auc_score(valid_y, XGB_best.predict_proba(valid_x)[:,1])

0.93232725610644

In [116]:
sub_xgb = XGB_best.predict_proba(test_x)[:,1]
sub_xgb

array([0.04218633, 0.00389896, 0.00393664, ..., 0.00333615, 0.01121438,
       0.04214636], dtype=float32)

In [119]:
sub_xgb = pd.DataFrame(sub_xgb, columns=['pred'])

sub_xgb = pd.concat([test_df.ID, sub_xgb], axis=1)

sub_xgb.to_csv('submission_xgb.csv', index_label=False, index=False)

In [107]:
xgb_ = xgb.XGBClassifier(n_jobs=4, n_estimators=200, learning_rate=.1, max_depth=4)

xgb_.fit(os_features, os_labels)
roc_auc_score(valid_y.values, xgb_.predict_proba(valid_x.values)[:,1])

0.9156338144958176

In [37]:
RFC = RandomForestClassifier(class_weight=class_weight, max_features=3, n_estimators=100,
                            min_samples_leaf=1, min_samples_split=3)

## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "min_samples_split": [2, 3],
              "min_samples_leaf": [1, 3],
              "n_estimators" :[100,300]}

gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", verbose = 1, n_jobs=4)

gsRFC.fit(train_x,train_y)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_, RFC_best

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   34.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  3.2min
[Parallel(n_jobs=4)]: Done 270 out of 270 | elapsed:  6.6min finished


(0.9007392359347667, RandomForestClassifier(bootstrap=False,
                        class_weight={0: 0.5662238325281803,
                                      1: 4.275075987841945},
                        criterion='gini', max_depth=None, max_features=3,
                        max_leaf_nodes=None, min_impurity_decrease=0.0,
                        min_impurity_split=None, min_samples_leaf=1,
                        min_samples_split=3, min_weight_fraction_leaf=0.0,
                        n_estimators=100, n_jobs=None, oob_score=False,
                        random_state=None, verbose=0, warm_start=False))

In [38]:
# with class_weight
roc_auc_score(valid_y, RFC_best.predict_proba(valid_x)[:,1])

0.9247010318521834

In [65]:
# without class_weight
roc_auc_score(valid_y, RFC_best.predict_proba(valid_x)[:,1])

0.927529086150826

In [40]:
DTC = DecisionTreeClassifier(class_weight=class_weight)

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", verbose = 1)

gsadaDTC.fit(train_x,train_y)

ada_best = gsadaDTC.best_estimator_

gsadaDTC.best_score_,ada_best

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:    9.6s finished


(0.8764742396027312, AdaBoostClassifier(algorithm='SAMME',
                    base_estimator=DecisionTreeClassifier(class_weight={0: 0.5662238325281803,
                                                                        1: 4.275075987841945},
                                                          criterion='entropy',
                                                          max_depth=None,
                                                          max_features=None,
                                                          max_leaf_nodes=None,
                                                          min_impurity_decrease=0.0,
                                                          min_impurity_split=None,
                                                          min_samples_leaf=1,
                                                          min_samples_split=2,
                                                          min_weight_fraction_leaf=0.0,
                             

In [41]:
# with class_weight
roc_auc_score(valid_y, ada_best.predict_proba(valid_x)[:,1])

0.6763775556981754

In [67]:
# without class_weight
roc_auc_score(valid_y, ada_best.predict_proba(valid_x)[:,1])

0.6931665579039904

In [21]:
ExtC = ExtraTreesClassifier()

## Search grid for optimal parameters
ex_param_grid = {"max_features": [1, 3],
              "min_samples_split": [2, 3],
              "min_samples_leaf": [1, 3],
              "n_estimators" :[300]}

gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", verbose = 1)

gsExtC.fit(train_x,train_y)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_, ExtC_best

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.6min finished


(0.8991591896619829,
 ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                      max_depth=None, max_features=3, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=3,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None, verbose=0,
                      warm_start=False))

In [23]:
# without class_weight
roc_auc_score(valid_y, ExtC_best.predict_proba(valid_x)[:,1])

0.9211764132855856

In [50]:
confusion_matrix(valid_y, ExtC_best.predict(valid_x))

array([[6611,  101],
       [ 632,  252]], dtype=int64)

## OverSample

In [91]:
oversampler = SMOTE(sampling_strategy=.5, random_state=0)
os_features, os_labels = oversampler.fit_sample(train_x, train_y)

In [92]:
os_features.shape

(23466, 16)

In [93]:
os_labels.shape

(23466,)

In [94]:
RFC = RandomForestClassifier(max_features=3, n_estimators=100,
                            min_samples_leaf=1, min_samples_split=3)
RFC.fit(os_features, os_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=3, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [108]:
roc_auc_score(valid_y.values, RFC.predict_proba(valid_x.values)[:,1])

0.9151478374654163